In [1]:
#!pip install celery

In [2]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Error: user_already_exists: myguest
Setting permissions for user "myguest" in vhost "/" ...


In [3]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a whoamI.py file 
You will need to run that code on the **worker** machines:

**"celery -A whoamI worker --loglevel=info --concurrency=3" on worker machine 1** <br>
**"celery -A whoamI worker --loglevel=info --concurrency=2" on worker machine 2** <br>
**"celery -A whoamI worker --loglevel=info --concurrency=4" on worker machine 3**

(The server machine can also be your worker machine, you will just need to run these in a terminal as new processes)

Then those processes/machines will become workers, and will be able to run the app task, i.e. in this case the echo function, whenever the broker requests it.

The server can ask the workers to run the echo function by these commands:
``` python
from test import echo
res = echo.delay('Python rocks!'); 
print(res)
res.result
```

In [4]:
from whoamI import echo

**Check the available worker machines** <br>
In the <code> !celery -A test status </code> command we use "test" because "test.py" runs on the workers

In [5]:
!celery -A whoamI status

->  celery@ip-172-31-32-7: OK
->  celery@ip-172-31-35-180: OK
->  celery@ip-172-31-46-167: OK

3 nodes online.


**Now let us run a series of jobs asynchronously on the workers.** 

In [6]:
res =[]
for iter in range(1,50):
    res.append(echo.delay('hello ' + str(iter)))

**Let us wait for the results with ".get()", and print them out**

In [7]:
for iter in range(len(res)):
    print(res[iter].get()) #let us wait for the results and print them out

Response from worker: 172.31.32.7 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 6979 message: hello 1 **
Response from worker: 172.31.35.180 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 12175 message: hello 2 **
Response from worker: 172.31.46.167 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 21687 message: hello 3 **
Response from worker: 172.31.32.7 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 6977 message: hello 4 **
Response from worker: 172.31.35.180 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 12172 message: hello 5 **
Response from worker: 172.31.46.167 process_name: ForkPoolWorker-4 process_index: 3 os_pid: 21690 message: hello 6 **
Response from worker: 172.31.32.7 process_name: ForkPoolWorker-3 process_index: 2 os_pid: 6980 message: hello 7 **
Response from worker: 172.31.35.180 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 12172 message: hello 8 **
Response from worker: 172.31.46.167 process_name: ForkPoolWorker-

Note that the job allocation looks like a round-robin among the different host machines for a while, but it is not always exactly like that.

In one instance, the same ip address and process id got tasks right after each other: <br>

Response from worker: 172.31.46.167 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 21684 message: hello 45 <br>
Response from worker: 172.31.46.167 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 21684 message: hello 46  <br>

**Let us run the echo function on the server instead of the workers:**

In [9]:
res=echo('running on the server')

172.31.46.167 process_name: MainProcess process_index: _none_ os_pid: 22333 message: running on the server **
